In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from cdec_maps import cdec

In [ ]:
import glob
import datetime
import pandas as pd

In [ ]:
r = cdec.Reader()

In [ ]:
cache_dir='cdec_cache'

In [ ]:
import os
if not os.path.exists(cache_dir): os.mkdir(cache_dir)

In [ ]:
mtime=os.path.getmtime(f'{cache_dir}/cdec.daily_stations.pkl')
mtime

In [ ]:
glob.glob('cdec_cache/station_meta_info.07K.*.pkl')

In [ ]:
delta=datetime.datetime.now()-datetime.datetime.fromtimestamp(mtime)

In [ ]:
delta.total_seconds()

In [ ]:
delta > pd.to_timedelta('1D') 

In [ ]:
pd.Timestamp.now()-pd.Timestamp.fromtimestamp(mtime) > pd.to_timedelta('1D')

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_pickle(f'{cache_dir}/cdec.daily_stations.pkl')

In [ ]:
dfdaily = r.read_daily_stations()
dfdaily.to_pickle(f'{cache_dir}/cdec.daily_stations.pkl')

In [ ]:
dfreal=r.read_realtime_stations()
dfreal.to_pickle(f'{cache_dir}/cdec.realtime_stations.pkl')

In [ ]:
dfsensors=r.read_sensor_list()
dfsensors.to_pickle(f'{cache_dir}/cdec.sensor_list.pkl')

In [ ]:
dfdaily

In [ ]:
import pandas as pd
pd.concat([dfreal.set_index('ID'),dfdaily.set_index('ID')], axis=1, join='inner').reset_index()

In [ ]:
dfdaily.merge(dfreal, on='ID', how='inner', suffixes=('_d', '_r'))

In [ ]:
dfreal

In [ ]:
dfdaily.set_index('ID').index.difference(dfreal.set_index('ID').index)

In [ ]:
dfreal.set_index('ID').index.difference(dfdaily.set_index('ID').index)

In [ ]:
dfall=dfdaily.merge(dfreal, on='ID', how='outer', suffixes=('_d', '_r'))
dfall

In [ ]:
dfall=dfall.set_index('ID')

In [ ]:
all_ids=[id for id in dfall.index]

In [ ]:
failed_ids=[]
for id in all_ids:
    try:
        dflist = r.read_station_meta_info(id)
        for i,df in enumerate(dflist):
            df.to_pickle(f'{cache_dir}/station_meta_info.{id}.{i}.pkl')
    except Exception as exc:
        print(f'Failed to read station meta info {id}')
        failed_ids.append(id)

In [ ]:
for id in failed_ids:
    try:
        dflist = r.read_station_meta_info(id)
        for i,df in enumerate(dflist):
            df.to_pickle(f'{cache_dir}/station_meta_info.{id}.{i}.pkl')
    except Exception as exc:
        print(f'Failed to read station meta info {id}')